In [1]:
cd ..

/home/taiamiti/Projects/micro-plastic/data_engineering


# Load dataset with tags and export tasks


copy export dataset from irene computer to `data/processed/remote_fiftyone/lot1_lot4_review`
add symlink to 

In [2]:
import fiftyone as fo
import fiftyone.brain as fob

import os
from pathlib import Path

import numpy as np

In [3]:
from fiftyone import ViewField as F

In [4]:
from PIL import Image

In [5]:
dataset_dir = "data/processed/remote_fiftyone/lot1_lot4_review/"

In [6]:
fo.list_datasets()

['aerial_view_windows_doors',
 'extra',
 'mp_dataset',
 'processed-dataset',
 'results',
 'side_view_windows_doors']

In [7]:
# dataset = fo.load_dataset("lot1_lot4_review")
# dataset.delete()

In [8]:
# The type of the dataset being imported
dataset_type = fo.types.FiftyOneDataset  # for example

dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name="lot1_lot4_review",
    
)

Importing samples...
 100% |███████████████| 4332/4332 [3.9s elapsed, 0s remaining, 1.6K samples/s]       
Import complete


In [9]:
dataset.count_sample_tags()

{'lot2-30-05-2023-tak_nacl': 206,
 'lot3-08-06-2023-benitiers': 448,
 're-annotate': 3410,
 'train': 1609,
 'test': 697,
 'lot2-30-05-2023-tak_nai': 634,
 'PartEntremelee': 5,
 'lot4-28-06-2023-sediments': 2658,
 'lot1-20-04-2023-sediments': 337,
 'unlabelled': 2020,
 'lot1-20-04-2023-benitiers': 49}

In [10]:
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [11]:
new_filepaths = [(fp.replace("C:\\Users\\igoder01\\Desktop\\remote_fiftyone\\", "")
  .replace("\\", "/")
  .replace("data/processed/remote_fiftyone/lot1_lot4_review/", "")
  .replace("semantic/", "")
 ) for fp in dataset.values("filepath")]

In [12]:
new_mask_paths = []
for fp in dataset.values("ground_truth.mask_path"):
    if fp is not None:
        new_mask_path = fp.replace("C:\\Users\\igoder01\\Desktop\\remote_fiftyone\\", 
                                   "/home/taiamiti/Projects/micro-plastic/data_engineering/").replace('\\', '/').replace("semantic/", "")
    else:
        new_mask_path = None
    new_mask_paths.append(new_mask_path)

In [13]:
new_mask_paths

['/home/taiamiti/Projects/micro-plastic/data_engineering/data/processed/generate_annotated_dataset/lot1-20-04-2023-benitiers/fields/ground_truth/2023-04-05_BENI_TAK_S1_3V_F1_D0_0007_COMP_500 pm.png',
 '/home/taiamiti/Projects/micro-plastic/data_engineering/data/processed/generate_annotated_dataset/lot1-20-04-2023-benitiers/fields/ground_truth/2023-04-05_BENI_TAK_S1_3V_F1_D0_0002_COMP_500 pm.png',
 '/home/taiamiti/Projects/micro-plastic/data_engineering/data/processed/generate_annotated_dataset/lot1-20-04-2023-benitiers/fields/ground_truth/2023-04-11_BENI_TAK_S1_3V_F2_D0_0032_COMP_500 pm.png',
 '/home/taiamiti/Projects/micro-plastic/data_engineering/data/processed/generate_annotated_dataset/lot1-20-04-2023-benitiers/fields/ground_truth/2023-04-11_BENI_TAK_S1_3V_F2_D0_0023_COMP_500 pm.png',
 '/home/taiamiti/Projects/micro-plastic/data_engineering/data/processed/generate_annotated_dataset/lot1-20-04-2023-benitiers/fields/ground_truth/2023-04-11_BENI_TAK_S1_3V_F2_D0_0006_COMP_500 pm.png',


In [14]:
dataset.set_values("filepath", new_filepaths)
dataset.set_values("ground_truth.mask_path", new_mask_paths)

In [15]:
from src.create_tasks import cluster_dataset, export_tasks, cluster_dataset_dbscan, visualize_clusters

dataset = cluster_dataset(dataset, n_clusters=15)

In [16]:
dataset.count_values("sample_type")

{'BENI': 497, 'SED': 3212, 'CSED': 623}

In [17]:
data_to_annotate = dataset.match_tags(['PartEntremelee', 're-annotate'])
data_to_annotate = data_to_annotate.match_tags(["unlabelled"], bool=False)
data_to_annotate = data_to_annotate.match(F("sample_type").contains_str("SED"))

In [20]:
embeddings = np.array(data_to_annotate.values("embeddings"))

# Compute 2D representation using pre-computed embeddings
results = fob.compute_visualization(
    data_to_annotate,
    embeddings=embeddings,
    num_dims=2,
    brain_key="image_embeddings",
    verbose=True,
    seed=51,
    min_dist=0.0,  # low value to pack points densely
    num_neighbors=20,  # put it low to get local structures alike or high to get global structure
    random_state=51
)

# Visualize image embeddings colored by time of day
plot = results.visualize(
    labels="cluster15",
    labels_title="cluster",
    axis_equal=True,
)
plot.show(height=512)

# plot = results.visualize(
#     labels="dataset_name",
#     labels_title="dataset_name",
#     axis_equal=True,
# )
# plot.show(height=512)

Ignoring unsupported parameters {'random_state'} for <class 'fiftyone.brain.visualization.UMAPVisualizationConfig'>
Generating visualization...
UMAP(min_dist=0.0, n_neighbors=20, random_state=51, verbose=True)
Mon Nov  6 15:00:54 2023 Construct fuzzy simplicial set
Mon Nov  6 15:00:55 2023 Finding Nearest Neighbors
Mon Nov  6 15:00:57 2023 Finished Nearest Neighbor Search
Mon Nov  6 15:00:59 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Mon Nov  6 15:01:02 2023 Finished embedding


FigureWidget({
    'data': [{'customdata': array(['64b37f7f02d2a387e1509380', '64b37f7f02d2a387e1509382',
                                   '64b37f7f02d2a387e1509383', ..., '64b19da81267afa45631f07c',
                                   '64b19da81267afa45631f081', '64b19da81267afa45631f08f'], dtype=object),
              'hovertemplate': ('<b>cluster: %{marker.color}</b' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([12, 11,  4, ..., 11, 11, 12]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction', 'title': {'text': 'cluster'}},
                         'colorscale': [[0.0, '#440154'], [0.1111111111111111,
                                        '#482878'], [0.2222222222222222,
                                        '#3e4989'], [0.3333333333333333,
                                        '#31688e'], [0.4444444444444444,
                                        '#26828e'], [0.5555555555555556,
                                        '#

In [21]:
session = fo.launch_app(view=data_to_annotate, auto=False)
session.open_tab()
# Attach plot to session
session.plots.attach(plot)

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [22]:
# session.open_tab()

## export annotation tasks for sed

In [24]:
from src.create_tasks import export_tasks

export_tasks(data_to_annotate, "data/processed/data_to_annotate/lot1_lot4_review_sed")

Directory 'data/processed/data_to_annotate/lot1_lot4_review_sed/00' already exists; export will be merged with existing files
 100% |█████████████████| 109/109 [8.4s elapsed, 0s remaining, 11.3 samples/s]      
 100% |███████████████████| 98/98 [7.1s elapsed, 0s remaining, 20.0 samples/s]      
 100% |███████████████████| 38/38 [1.6s elapsed, 0s remaining, 26.5 samples/s]      
 100% |███████████████████| 60/60 [3.7s elapsed, 0s remaining, 18.5 samples/s]      
 100% |███████████████████| 66/66 [6.0s elapsed, 0s remaining, 12.0 samples/s]      
 100% |███████████████████| 57/57 [3.1s elapsed, 0s remaining, 26.6 samples/s]      
 100% |█████████████████| 128/128 [7.7s elapsed, 0s remaining, 24.7 samples/s]      
 100% |███████████████████| 61/61 [2.4s elapsed, 0s remaining, 26.3 samples/s]      
 100% |███████████████████| 44/44 [2.0s elapsed, 0s remaining, 24.2 samples/s]         
 100% |███████████████████| 74/74 [4.3s elapsed, 0s remaining, 18.8 samples/s]      
 100% |██████████████

## export annotation tasks for beni

In [ ]:
data_to_annotate2 = dataset.match_tags(['PartEntremelee', 're-annotate'])
data_to_annotate2 = data_to_annotate2.match(F("sample_type").contains_str("BENI"))
export_tasks(data_to_annotate, "data/processed/data_to_annotate/lot1_lot4_review_beni")